In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

from src.data.loader import load_interactions, load_games
from src.evaluation.splitter import split_train_in_out
from src.evaluation.evaluator import evaluate_model
from src.models.ease import EASE
from src.novelty.distance import (
    build_genre_similarity_matrix,
    build_genre_distance_matrix,
)
import numpy as np
from src.pipelines.save import save_submission
from src.config import LAMBDA_REG, TOP_K, N_EVAL_USERS, SEED

# PART 1: OFFLINE EVALUATION

In [ ]:
# 1. Load data
train = load_interactions(train=True)
games = load_games()

# maps item_id -> publisher (needed for publisher Gini)
publisher_mapper = games.set_index("item_id")["publisher"]

In [ ]:
# 2. Local split (Codabench-like: 1 holdout per user)
train_in_full, train_out_full = split_train_in_out(train, seed=SEED)

In [ ]:
# 3. Sample a subset of users for faster offline eval
all_users = train_out_full["user_id"].unique()          # only users with a holdout
rng = np.random.default_rng(SEED)
n_eval = min(N_EVAL_USERS, len(all_users))
sample_users = rng.choice(all_users, size=n_eval, replace=False)

In [ ]:
train_in = train_in_full[train_in_full["user_id"].isin(sample_users)].reset_index(drop=True)
train_out = train_out_full[train_out_full["user_id"].isin(sample_users)].reset_index(drop=True)

print(f"Offline eval on {n_eval} users "
      f"(train_in rows = {len(train_in)}, train_out rows = {len(train_out)})")

In [ ]:
# 4. Build distance and similarity once (all items)
item_similarity = build_genre_similarity_matrix(games)
item_distance = build_genre_distance_matrix(games)

In [ ]:
# 5. Fit EASE on *fold-in* for these users and get recommendations
model = EASE(lambda_reg=LAMBDA_REG)

# train_in = history (fold-in) for sampled users
# test_in = same df; we score these users
recs_offline = model.recommend(train_in, train_in, top_k=TOP_K)

In [ ]:
# 6. Evaluate offline baseline on the sampled users
metrics_offline = evaluate_model(
    recs_offline,
    train_in,
    train_out,
    publisher_mapper=publisher_mapper,
    item_similarity=item_similarity,
    item_distance=item_distance,
)
print(f"Offline EASE baseline (lambda_reg = {LAMBDA_REG}) on {n_eval} users")
print(metrics_offline)

# PART 2: ONLINE EVALUATION (codabench)

In [22]:
# 1. Load full training and official test fold-in
train_full = load_interactions(train=True)
test_in = load_interactions(train=False)
lambda_reg = 400

In [23]:
# 2. New model instance, same lambda
model_cb = EASE(lambda_reg=lambda_reg)

In [24]:
# 3. train_full is used to fit, test_in defines which users we score
recs_codabench = model_cb.recommend(train_full, test_in, top_k=TOP_K)


[generate_recommendations] starting
  train_df rows: 2293985
  test_in_df rows: 448211
  users in train_df: 54315
  users in test_in_df: 13579

[build_interaction_matrix] starting
  rows in interactions: 2293985
  unique users:         54315
  unique items:         8368
  user_id range:        0 to 68402
  item_id range:        0 to 8522
  matrix users (rows):  68403
  matrix items (cols):  8523
  nnz interactions:     2267389
  density:              0.003889
  avg items per user:   33.15
  avg users per item:   266.03
[build_interaction_matrix] done in 0.11 s

  model.W is None, fitting EASE on X_train

[EASE.fit] starting
  X shape:    (68403, 8523)
  X nnz:      2267389
  avg items per user: 33.15  avg users per item: 266.03
  lambda_reg: 400.0
  built G (X^T X) with shape (8523, 8523) in 1.67 s
  G diag before reg - min: 0.00, max: 35340.00, mean: 275.46
  G diag after reg  - min: 400.00, max: 35740.00, mean: 675.46
  inverted G in 9.53 s
  W nonzeros: 70015056
  W density:  0.963

In [25]:
# Quick sanity check
print(recs_codabench.head(n=40))

    user_id  item_id     score  rank
0         4      307  1.392060     1
1         4     8213  1.123224     2
2         4     1043  1.086776     3
3         4      450  1.008484     4
4         4      658  0.958766     5
5         4     5888  0.928391     6
6         4     8327  0.910941     7
7         4     7996  0.908962     8
8         4        0  0.816616     9
9         4      344  0.812480    10
10        4      252  0.794504    11
11        4     8017  0.786572    12
12        4     1548  0.695928    13
13        4      328  0.681410    14
14        4       90  0.669683    15
15        4      697  0.667628    16
16        4      539  0.665983    17
17        4      647  0.664863    18
18        4      579  0.658385    19
19        4     7711  0.657440    20
20        6     1072  0.964183     1
21        6     8278  0.834352     2
22        6     7711  0.686152     3
23        6      669  0.643263     4
24        6       46  0.597305     5
25        6     4461  0.578455     6
2

In [26]:
# 7. Save submission CSV (this writes submission_name.csv in notebooks or pipelines location)
submission_name = f"ease_baseline_lambda_{lambda_reg}"
save_submission(recs_codabench, submission_name)
print(f"Saved Codabench submission as {submission_name}.csv")

Saved CSV: /home/sunaj/Desktop/novelty-aware-recommenders/results/submissions/ease_baseline_lambda_400.csv
Saved ZIP for Codabench: /home/sunaj/Desktop/novelty-aware-recommenders/results/submissions/ease_baseline_lambda_400.csv.zip
Saved Codabench submission as ease_baseline_lambda_400.csv
